In [ ]:
# prompt: i want to deploy this on streamlit

import streamlit as st
import pandas as pd
from docx import Document
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os # Import os for checking file existence

# Define the device for model computation
dev = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device for generation: {dev}")

# --- Load Embedding Model ---
@st.cache_resource # Cache the embedding model
def load_embedding_model():
    return SentenceTransformer('all-MiniLM-L6-v2')

embedding_model = load_embedding_model()

# --- Load Model for Generation ---
@st.cache_resource # Cache the generative model and tokenizer
def load_generative_model(model_name="ByteDance-Seed/Seed-Coder-8B-Reasoning", device="cpu"):
    try:
        # Configure quantization (adjust based on your hardware and needs)
        # Using 8-bit loading for potential memory savings on GPU
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True if device == 'cuda' else False,
            # load_in_4bit=False,
            # bnb_4bit_compute_dtype=torch.float16,
            # bnb_4bit_quant_type="nf4",
            # bnb_4bit_use_double_quant=False,
        )

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)

        # Move model to device if not using 8-bit or 4-bit loading
        if not (quantization_config.load_in_8bit or getattr(quantization_config, 'load_in_4bit', False)):
            model.to(device)

        return tokenizer, model
    except Exception as e:
        st.error(f"Error loading the generative model: {e}")
        st.warning("Please ensure the model name is correct and necessary libraries (like bitsandbytes) are installed if using 8-bit loading.")
        return None, None

tokenizer, model = load_generative_model(device=dev)

# --- Data Loading Functions ---
# Ensure these functions can load from paths accessible by the Streamlit app
# For Streamlit Cloud, you might need to upload these files or access them differently.
# For local deployment, ensure the paths are correct relative to where you run the script.

def load_sts_data(csv_path):
    if not os.path.exists(csv_path):
        st.error(f"STS CSV file not found at {csv_path}")
        return None
    try:
        return pd.read_csv(csv_path, encoding='latin-1')
    except Exception as e:
        st.error(f"Error loading STS data from {csv_path}: {e}")
        return None

def load_past_paper(docx_path):
    if not os.path.exists(docx_path):
        st.error(f"Past Papers DOCX file not found at {docx_path}")
        return None
    try:
        doc = Document(docx_path)
        qa_pairs = []
        # Assuming the questions and answers are in the first table
        if not doc.tables:
             st.warning(f"No tables found in {docx_path}. Cannot load QA pairs.")
             return []

        table = doc.tables[0]
        rows = [cell.text.strip() for row in table.rows for cell in row.cells if cell.text.strip()]
        for i in range(0, len(rows), 2):
            question = rows[i]
            answer = rows[i+1] if i+1 < len(rows) else ''
            qa_pairs.append({'question': question, 'answer': answer})
        return qa_pairs
    except Exception as e:
        st.error(f"Error loading Past Papers data from {docx_path}: {e}")
        return None


# --- Generate Response Using the loaded LLM ---
def generate_llm_response(query, retrieved_answer, dev):
    if model is None or tokenizer is None:
        return "Generative model not loaded properly."

    prompt = f"You are an intelligent MDCAT assistant. Use the following info to answer the user's question.\n\nQuestion: {query}\n\nRetrieved Info: {retrieved_answer}\n\nAnswer:"
    try:
        # Move input tensors to the specified device
        inputs = tokenizer(prompt, return_tensors="pt").to(dev)

        # Generate response using the generative model
        # Use `input_ids` and `attention_mask` from inputs
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=256,
            do_sample=True,
            top_k=50,
            temperature=0.7
        )
        # Decode the generated tokens, skipping the input prompt tokens
        response = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
        return response.strip()
    except Exception as e:
        st.error(f"Error during LLM response generation: {e}")
        return "An error occurred during response generation."

# --- RAG Pipeline with LLM Identification ---
def ask_mdcat_assistant_smart(query, sts_data, past_papers, embedding_model, dev, similarity_threshold=0.7):
    """
    Answers a query by first checking STS and Past Papers datasets
    using semantic search. If a confident match is found in either,
    it retrieves the relevant answer and uses the LLM to generate a response.
    If no confident match is found in either, the LLM answers on its own.
    """
    sts_match = None
    past_papers_match = None
    sts_best_score = -1
    past_papers_best_score = -1

    # Prepare STS data for semantic search
    sts_qa_pairs = []
    if sts_data is not None:
        sts_qa_pairs = [{'question': q, 'answer': a} for q, a in zip(sts_data['query'].tolist(), sts_data['answer'])]

    # Semantic search in STS data
    if sts_qa_pairs:
        try:
            sts_questions = [item['question'] for item in sts_qa_pairs]
            sts_question_embeddings = embedding_model.encode(sts_questions, convert_to_tensor=True)
            query_embedding = embedding_model.encode(query, convert_to_tensor=True)
            sts_cos_scores = util.cos_sim(query_embedding, sts_question_embeddings)
            sts_best_idx = sts_cos_scores.argmax()
            sts_best_score = sts_cos_scores[0][sts_best_idx].item()

            if sts_best_score >= similarity_threshold:
                sts_match = sts_qa_pairs[sts_best_idx]
                st.info(f"Confident match found in STS data with score: {sts_best_score:.4f}")
        except Exception as e:
            st.error(f"Error during STS semantic search: {e}")


    # Semantic search in Past Papers data
    if past_papers:
        try:
            past_papers_questions = [item['question'] for item in past_papers]
            past_papers_question_embeddings = embedding_model.encode(past_papers_questions, convert_to_tensor=True)
            query_embedding = embedding_model.encode(query, convert_to_tensor=True) # Re-encode query
            past_papers_cos_scores = util.cos_sim(query_embedding, past_papers_question_embeddings)
            past_papers_best_idx = past_papers_cos_scores.argmax()
            past_papers_best_score = past_papers_cos_scores[0][past_papers_best_idx].item()

            if past_papers_best_score >= similarity_threshold:
                past_papers_match = past_papers[past_papers_best_idx]
                st.info(f"Confident match found in Past Papers data with score: {past_papers_best_score:.4f}")
        except Exception as e:
            st.error(f"Error during Past Papers semantic search: {e}")


    retrieved_info = None
    source = "LLM (General Knowledge)" # Default source if no confident match

    # Determine which source to use based on confidence score
    if sts_match and past_papers_match:
        if sts_best_score > past_papers_best_score:
            retrieved_info = sts_match['answer']
            source = "STS Data"
        else:
            retrieved_info = past_papers_match['answer']
            source = "Past Papers Data"
    elif sts_match:
        retrieved_info = sts_match['answer']
        source = "STS Data"
    elif past_papers_match:
        retrieved_info = past_papers_match['answer']
        source = "Past Papers Data"

    st.write(f"Using source: {source}")

    if retrieved_info:
        st.write("Generating LLM response using retrieved info...")
        response = generate_llm_response(query, retrieved_info, dev)
    else:
        st.write("No confident match found in datasets. LLM answering directly...")
        if model is None or tokenizer is None:
             return "Generative model not loaded properly."

        # Construct a prompt for the LLM to answer on its own
        direct_prompt = f"You are an intelligent MDCAT assistant. Answer the following question:\n\nQuestion: {query}\n\nAnswer:"
        try:
            inputs = tokenizer(direct_prompt, return_tensors="pt").to(dev)

            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=256,
                do_sample=True,
                top_k=50,
                temperature=0.7
            )
            response = tokenizer.decode(outputs[0][inputs.input_ids.shape[-1]:], skip_special_tokens=True)
            response = response.strip()
        except Exception as e:
            st.error(f"Error during direct LLM generation: {e}")
            response = "An error occurred during direct response generation."


    return response

# --- Streamlit UI ---
st.title("MDCAT Assistant")
st.write("Ask me anything about the MDCAT!")

# --- Data File Paths ---
# IMPORTANT: Update these paths to where your files are located relative to your Streamlit app.
# If deploying to Streamlit Cloud, you'll need to upload these files or use a different storage method.
sts_csv_path = 'mdcat_queries.csv' # Example: Assuming the file is in the same directory
past_papers_docx_path = 'past_tests.docx' # Example: Assuming the file is in the same directory

# Load data
sts_data = load_sts_data(sts_csv_path)
past_papers = load_past_paper(past_papers_docx_path)

# Check if data and models loaded successfully
if sts_data is None or past_papers is None or model is None or tokenizer is None:
    st.error("Required data or models failed to load. The assistant cannot function.")
else:
    user_query = st.text_input("Enter your question:")

    if user_query:
        with st.spinner("Thinking..."):
            response = ask_mdcat_assistant_smart(user_query, sts_data, past_papers, embedding_model, dev)
        st.subheader("Assistant Response:")
        st.write(response)

st.markdown("---")
st.write("Note: This assistant uses a RAG pipeline and a large language model to answer questions.")

